# Proyecto: Análisis del mercado inmobiliario de la Ciudad de Buenos Aires

Mentor: Javier Lezama.

javitolez@gmail.com

En este proyecto vamos a trabajar con un dataset de propiedades en venta publicadas en el portal www.Properati.com.ar.

# El dataset: propiedades en venta en Ciudad de Buenos Aires.

En este proyecto, trabajaremos con una muestra del conjunto de datos de propiedades en venta mencionado. En este dataset, cada fila es una propiedad en venta. 

#Los atributos

A continuación vamos a describir los atributos que consideramos en esta muestra:
* id: id de la propiedad
* created_on: fecha en la que la propiedad ingresó al sitio
* operation: alquiler (rent) o venta (sell)
* property_type: tipo de propiedad (casa, departamento, ph, etcétera)
* place_with_parent_names: nombre del lugar donde se encuentra la propiedad según el publicador
* lat-lon: coordenadas concatenadas
* lat: latitud
* lon: longitud
* price: precio en la moneda especificada en currency
* currency: divisa en la que está expresada la publicación
* price_aprox_usd: precio aproximado en dólares estadounidenses
* surface_total_in_m2: superficie total (en metros cuadrados)
* surface_covered_in_m2: superficie cubierta (en metros cuadrados)
* price_usd_per_m2: precio por metro cuadrado en dólares (precio dólares / superficie)
* floor: número de piso (si corresponde)
* rooms: cantidad de ambientes
* expenses: expensas (si corresponde)
* barrio: barrio según cartografía oficial
* properati_url: url de la publicación en Properati

Objetivos del practico introducción al aprendizaje automático:

El objetivo de este practico es trabajar con el mismo dateset trabajado en el práctico dos, específicamente el objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

Para eso trabajaremos con el dataset resultante del practico dos para armar nuestros primeros modelos para predecir el precio de las propiedades en dólares.

# Manos a la obra

Partiremos con el dataset resultante del práctico dos. El práctico está basado en el práctico final de la materia introducción al aprendizaje automatico del año 2018.

In [115]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib.colors import ListedColormap
from sklearn.datasets import load_boston, load_breast_cancer, load_iris
from sklearn.linear_model import LinearRegression, LogisticRegression, Perceptron, Ridge
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

np.random.seed(1234)  # Setup seed to be more deterministic

%matplotlib inline

In [116]:
df_prop = pd.read_csv('df_bin.csv')
df_prop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7716 entries, 0 to 7715
Data columns (total 79 columns):
Unnamed: 0                                                                       7716 non-null int64
created_on                                                                       7716 non-null object
operation                                                                        7716 non-null object
property_type                                                                    7716 non-null int64
place_with_parent_names                                                          7716 non-null object
lat-lon                                                                          7716 non-null object
lat                                                                              7716 non-null float64
lon                                                                              7716 non-null float64
price_aprox_usd                                                              

# Ejercicio 1: División de datos en conjuntos de entrenamiento y evaluación.

La primer tarea consiste en dividir el conjunto de datos cargados en el apartado anterior en conjuntos de entrenamiento (o training) y evaluación (o test).

Se define un dataframe con los features que se consideran importantes para la definición del precio de la propiedad

In [117]:
df_prop_gt = df_prop[['surface_total_in_m2','barrio_cod','property_type','price_aprox_usd']]
df_prop_gt.head()

,surface_total_in_m2,barrio_cod,property_type,price_aprox_usd
0,130.0,20,1,340000.00
1,70.0,20,1,180000.00
2,532.0,20,2,1200000.00
3,72.0,20,0,219000.00
4,40.0,20,1,119124.77


Se separa el set de datos tomando el 80% para entrenamiento y el 20% para validación

In [118]:
#Utilizamos aproximadamente 90% de los datos para entrenamiento y 20% para validación
#entradas = len(df_prop)
#data_div = round(entradas*0.9)

#shuff_data = np.random.permutation(506)
#shuff_train = shuff_data[:data_div]
#shuff_val = shuff_data[data_div:]

X = df_prop_gt.drop('price_aprox_usd', axis=1)
y = df_prop_gt.price_aprox_usd

from sklearn.model_selection import train_test_split
#Random state sirve para sembrar una "Seed" de como se distribuyen los datos para el algoritmo
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Necesario para poder hacer un regresor por feature
#features_names = df_prop.columns
#feature_map = {feature: idx for idx, feature in enumerate(features_names)}


# Ejercicio 2: Elección de un modelo de regresión.

https://gist.github.com/lprowell/81ab813d84096290c5a833a8c7731355

Basándose en lo visto en el teórico escojan y justifiquen un modelo de aprendizaje automático de regresión, para predecir el precio de las propiedades en dólares. Selección de regulizador, selección de la función de coste, etc.

In [119]:
# Entrenamos un clasificador utilizando sólo ese atributo sobre el conjunto de entrenamiento (X_train, y_train)
#model = LinearRegression()
model = Ridge(alpha=0.01)
model.fit(X_train, y_train)

# Evaluamos el desempeño del clasificador utilizando la media del error cuadrado (MSE o mean squared error)
# sobre el conjunto de datos de entrenamiento (X_train, y_train) y lo comparamos con el de validación (X_val, y_val)
# Mientras más cercano a cero mejor
print('Media del error cuadrado para entrenamiento: %.2f' % 
      mean_squared_error(y_train, model.predict(X_train)))
#print('Media del error cuadrado para validación: %.2f' %
#      mean_squared_error(yy, model.predict(XX)))

Media del error cuadrado para entrenamiento: 1386185771.41


# Ejercicio 3: Selección de hiperparámetros.

Utilizando búsqueda exhaustiva (grid search) con 5-fold cross-validation y utilizando como métrica el área bajo la curva de ROC (o ROC-AUC), hagan una selección de los mejores hiperparámetros para su conjunto de datos y el modelo que hayan elegido en el apartado anterior.

In [93]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

### GridSearchCV Regresión Lineal

In [122]:
params={'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]}
rdg_reg = Ridge()

model = GridSearchCV(rdg_reg,params,cv=2,verbose = 1, scoring = 'neg_mean_squared_error')         
model.fit(X_train, y_train)

pd.DataFrame(model.cv_results_)

#print("# Exploración de hiperparámetros para función de coste \"%s\"" % loss, end="\n\n")
    
#print("Mejor conjunto de parámetros:")
#print(model.best_params_, end="\n\n")



Fitting 2 folds for each of 13 candidates, totalling 26 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:    0.1s finished
C:\Users\Gabriel H\.conda\envs\diplodatos\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Gabriel H\.conda\envs\diplodatos\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Gabriel H\.conda\envs\diplodatos\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be a

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,mean_train_score,std_train_score
0,0.003944,6.839037e-04,0.001618,0.000619,25,{'alpha': 25},-1.309485e+09,-1.475591e+09,-1.392538e+09,8.305284e+07,13,-1.466884e+09,-1.301259e+09,-1.384071e+09,8.281262e+07
1,0.002894,2.894163e-03,0.003248,0.000762,10,{'alpha': 10},-1.309486e+09,-1.475589e+09,-1.392538e+09,8.305164e+07,12,-1.466884e+09,-1.301259e+09,-1.384071e+09,8.281260e+07
2,0.001553,5.550385e-04,0.003906,0.000104,4,{'alpha': 4},-1.309486e+09,-1.475589e+09,-1.392538e+09,8.305115e+07,11,-1.466884e+09,-1.301259e+09,-1.384071e+09,8.281260e+07
3,0.004162,1.666546e-04,0.002495,0.001499,2,{'alpha': 2},-1.309487e+09,-1.475589e+09,-1.392538e+09,8.305098e+07,10,-1.466884e+09,-1.301259e+09,-1.384071e+09,8.281260e+07
4,0.003614,1.620173e-03,0.002523,0.001526,1,{'alpha': 1.0},-1.309487e+09,-1.475588e+09,-1.392538e+09,8.305090e+07,9,-1.466884e+09,-1.301259e+09,-1.384071e+09,8.281260e+07
5,0.002587,1.589537e-03,0.000580,0.000580,0.8,{'alpha': 0.8},-1.309487e+09,-1.475588e+09,-1.392538e+09,8.305089e+07,8,-1.466884e+09,-1.301259e+09,-1.384071e+09,8.281260e+07
6,0.003544,4.632473e-04,0.002497,0.001499,0.5,{'alpha': 0.5},-1.309487e+09,-1.475588e+09,-1.392538e+09,8.305086e+07,7,-1.466884e+09,-1.301259e+09,-1.384071e+09,8.281260e+07
7,0.003995,2.384186e-07,0.004068,0.000074,0.3,{'alpha': 0.3},-1.309487e+09,-1.475588e+09,-1.392538e+09,8.305084e+07,6,-1.466884e+09,-1.301259e+09,-1.384071e+09,8.281260e+07
8,0.004001,3.933907e-06,0.000000,0.000000,0.2,{'alpha': 0.2},-1.309487e+09,-1.475588e+09,-1.392538e+09,8.305084e+07,5,-1.466884e+09,-1.301259e+09,-1.384071e+09,8.281260e+07
9,0.003590,4.153252e-04,0.000726,0.000726,0.1,{'alpha': 0.1},-1.309487e+09,-1.475588e+09,-1.392538e+09,8.305083e+07,4,-1.466884e+09,-1.301259e+09,-1.384071e+09,8.281260e+07


### GridSearchCV Regresión Polinomial

# Ejercicio 4: Métricas sobre el conjunto de evaluación

Una vez encontrados los mejores hiperparámetros para el modelo seleccionado en los apartados anteriores se evalúa el modelo final entrenado sobre el conjunto de datos de evaluación seleccionado en el ejercicio 1. Pueden utilizar las métricas que crean convenientes.

# Informe final

Presentar un informe final con los datos inferidos del dataset utilizando herramientas gráficas y escritas teniendo en cuenta los destinatarios. En este caso colegas de la diplomatura para preparar el informe con un background adecuado al destinatario.